Importing Statements

In [1]:
%pip install datasets
%pip install accelerate -U
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling, GPT2Tokenizer, GPT2LMHeadModel, pipeline
import datasets
from datasets import load_dataset, list_datasets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Creating Tokenizer and Model from gpt2

In [2]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2')
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Getting data from csv files

In [12]:
dataset = load_dataset("csv", data_files=["/Users/jonathanpang/desktop/cs505/project/ClassesCSV.csv"])

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Tokenizing the data

In [22]:
def encode(batch): 
    return tokenizer([x.strip('\n\r') if (x is not None) else "" for x in batch['description']], truncation=True, padding=True)
    # ret = []
    # for x in batch['description']:
    #     if (x is not None):
    #         ret.append(x.strip('\n\r'))
    # return tokenizer(ret, truncation=True, padding=True)

processed = dataset.map(encode, batched=True, batch_size=len(dataset))
processed.set_format('torch', columns=['input_ids', 'attention_mask'])

Map:   0%|          | 0/18721 [00:00<?, ? examples/s]

AttributeError: 'NoneType' object has no attribute 'strip'

Creating Trainer for data

In [ ]:
training_args = TrainingArguments(
    output_dir='/Users/jonathanpang/desktop/cs505/project/content',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=processed,
)

Training the data

In [ ]:
trainer.train()